# DCOPF

In [28]:
using JuMP, HiGHS, Ipopt, Plots

┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1664
┌ Warning: backend `GR` is not installed.
└ @ Plots C:\Users\viniciusjusten\.julia\packages\Plots\M4dfL\src\backends.jl:37
[ Info: GR


In [2]:
Base.@kwdef mutable struct DCOPFModel
    optimizer::JuMP.Model = Model()
    var::Dict{String, Any} = Dict{String, Any}()
    con::Dict{String, Any} = Dict{String, Any}()
    expr::Dict{String, Any} = Dict{String, Any}()
    cuts::Dict{Tuple{String, Int}, Any} = Dict{Tuple{String, Int}, Any}()
    obj_exp::JuMP.AffExpr = zero(JuMP.AffExpr)
end

Base.@kwdef mutable struct DCOPFInputs
    phase_reference::Int = 1
    Ybus::Matrix{Complex{Float64}} = Complex{Float64}[;;]
    min_flow::Matrix{Float64} = Float64[;;]
    max_flow::Matrix{Float64} = Float64[;;]
    min_generation::Vector{Float64} = Float64[]
    max_generation::Vector{Float64} = Float64[]
    generation_cost::Vector{Float64} = Float64[]
    demand::Vector{Float64} = Float64[]
    consider_losses::Bool = false
    max_iteration::Int = 1
    tolerance::Float64 = 1e-4
end

Base.@kwdef mutable struct DCOPFResults
    # tuple is name and iteration
    var::Dict{Tuple{String, Float64}, Any} = Dict{Tuple{String, Float64}, Any}()
    expr::Dict{Tuple{String, Float64}, Any} = Dict{Tuple{String, Float64}, Any}()
    obj_exp::JuMP.AffExpr = zero(JuMP.AffExpr)
end

DCOPFResults

In [3]:
function variableref(args...)
    return Array{JuMP.VariableRef, length(args)}(undef, args...)
end
function constraintref(args...)
    return Array{JuMP.ConstraintRef, length(args)}(undef, args...)
end
function expressionref(args...)
    return Array{JuMP.AffExpr, length(args)}(undef, args...)
end

function save_all_results!(model::DCOPFModel, results::DCOPFResults, current_iteration::Int)
    for (key, group) in model.var
        var_values = fill(NaN, size(group))
        for i in eachindex(group)
            if isassigned(group, i)
                var_values[i] = JuMP.value(group[i])
            end
        end
        results.var[key, current_iteration] = var_values
    end
    for (key, group) in model.expr
        expr_values = fill(NaN, size(group))
        for i in eachindex(group)
            if isassigned(group, i)
                expr_values[i] = JuMP.value(group[i])
            end
        end
        results.expr[key, current_iteration] = expr_values
    end
    return nothing
end

function in_keys(dict::Dict, element::Any)
    for k in keys(dict)
        if element in k
            return true
        end
    end
    return false
end

in_keys (generic function with 1 method)

In [4]:
function linear_flow!(model::DCOPFModel, inputs::DCOPFInputs)
    optimizer_model = model.optimizer
    
    Ybus = inputs.Ybus
    Gbus = real(Ybus)
    Xbus = imag(Ybus.^(-1))
    num_bus = size(Ybus)[1]
    
    phase_reference = inputs.phase_reference
    demand = inputs.demand
    min_flow = inputs.min_flow
    max_flow = inputs.max_flow
    max_generation = inputs.max_generation
    min_generation = inputs.min_generation
    
    if !inputs.consider_losses
        power_loss = zeros(AffExpr, num_bus, num_bus)
        model.expr["PowerLoss"] = power_loss
    else
        power_loss_var = variableref(num_bus, num_bus)
        power_loss = zeros(AffExpr, num_bus, num_bus)
        for bus_from in 1:num_bus, bus_to in 1:num_bus
            if bus_from != bus_to
                power_loss_var[bus_from, bus_to] = @variable(
                    optimizer_model,
                    lower_bound = 0.0,
                )
                power_loss[bus_from, bus_to] = power_loss_var[bus_from, bus_to]
            end
        end
        model.var["PowerLossVar"] = power_loss_var
        model.expr["PowerLoss"] = power_loss
    end
    
    ### create variables
    phase = variableref(num_bus)
    generation = variableref(num_bus)
    
    ### create constraints
    load_balance = constraintref(num_bus)
    flow_lower_bound = constraintref(num_bus, num_bus)
    flow_upper_bound = constraintref(num_bus, num_bus)
    
    ### create expressionss
    flow = zeros(AffExpr, num_bus, num_bus)
    power_injection = expressionref(num_bus)
    
    ### define variables
    for bus in 1:num_bus
        generation[bus] = @variable(
            optimizer_model,
            lower_bound = min_generation[bus],
            upper_bound = max_generation[bus]
        )
        if bus == phase_reference
            phase[bus] = @variable(
                optimizer_model,
                lower_bound = 0,
                upper_bound = 0
            )
        else
            phase[bus] = @variable(
                optimizer_model
            )
        end
    end
    # save variables
    model.var["Phase"] = phase
    model.var["Generation"] = generation
    
    ### define expressions
    for bus_from in 1:num_bus, bus_to in 1:num_bus
        if bus_from == bus_to
            continue
        end
        flow[bus_from, bus_to] = @expression(
            optimizer_model,
            (phase[bus_from] - phase[bus_to])/Xbus[bus_from, bus_to]
        )
    end
    for bus in 1:num_bus
        power_injection[bus] = @expression(
            optimizer_model,
            sum(flow[bus, :])    
        )
    end
    # save expressions
    model.expr["Flow"] = flow
    model.expr["PowerInjection"] = power_injection
    
    ### define constraints
    for bus in 1:num_bus
        load_balance[bus] = @constraint(
            optimizer_model,
            generation[bus] == demand[bus] + power_injection[bus] + sum(power_loss[bus, :])/2
        )
    end
    for bus_from in 1:num_bus, bus_to in 1:num_bus
        flow_upper_bound[bus_from, bus_to] = @constraint(
            optimizer_model,
            flow[bus_from, bus_to] <= max_flow[bus_from, bus_to]
        )
        flow_lower_bound[bus_from, bus_to] = @constraint(
            optimizer_model,
            flow[bus_from, bus_to] >= min_flow[bus_from, bus_to]
        )
    end
    if inputs.consider_losses
        # creates epigraphs with power loss cuts
        loss_cuts!(model, inputs)
    end
    # save constraints
    model.con["LoadBalance"] = load_balance
    model.con["FlowLowerBound"] = flow_lower_bound
    model.con["FlowUpperBound"] = flow_upper_bound
    
end

function loss_cuts!(model::DCOPFModel, inputs::DCOPFInputs)
    Ybus = inputs.Ybus
    num_bus = size(Ybus)[1]
    phase = model.var["Phase"]
    for (k, group) in model.cuts
        if "PowerLoss" in k
            for bus_from in 1:num_bus, bus_to in 1:num_bus
                @constraint(
                    model.optimizer,
                    model.expr["PowerLoss"][bus_from, bus_to] >= 
                        [phase[bus_from] - phase[bus_to], 1]' * group[bus_from, bus_to]
                )
            end
        end
    end
end

function new_loss_cut!(
    model::DCOPFModel,
    inputs::DCOPFInputs,
    results::DCOPFResults,
    current_iteration::Int
)
    Ybus = inputs.Ybus
    num_bus = size(Ybus)[1]

    phase = results.var["Phase", current_iteration]
    loss_cut = Array{Vector{Float64}, 2}(undef, num_bus, num_bus)
    loss = zeros(num_bus, num_bus)

    for bus_from in 1:num_bus, bus_to in 1:num_bus
        phase_difference = phase[bus_from] - phase[bus_to]
        conductance = -real(Ybus)[bus_from, bus_to]
        loss_derivative = transmission_loss_derivative(phase_difference, conductance)
        loss[bus_from, bus_to] = transmission_loss(phase_difference, conductance)
        # cut is a vector [a, b] where y = a*x + b
        loss_cut[bus_from, bus_to] = [loss_derivative, loss[bus_from, bus_to] - (loss_derivative * phase_difference)]
    end

    if maximum(abs.(loss - results.expr["PowerLoss", current_iteration])) <= inputs.tolerance
        return :stop
    else
        model.cuts["PowerLoss", current_iteration] = loss_cut
        return nothing
    end
end

function transmission_loss(phase_difference::Float64, conductance::Float64)
    return conductance * phase_difference^2
end

function transmission_loss_derivative(phase_difference::Float64, conductance::Float64)
    return 2 * conductance * phase_difference
end

transmission_loss_derivative (generic function with 1 method)

## Rodadas
### Caso sem perdas

In [6]:
model = DCOPFModel()
model.optimizer = Model(HiGHS.Optimizer)

inputs = DCOPFInputs()
inputs.Ybus = [ 22.5im -10im -12.5im; -10im 30im -20im; -12.5im -20im 32.5im ]
inputs.demand = [0, 0, .8]
inputs.min_generation = zeros(3)
inputs.max_generation = [.5, .5, 0]
max_flow = [0 .1 .5; .1 0 .5; .5 .5 0]
inputs.min_flow = -max_flow
inputs.max_flow = max_flow
cost = [.5, 1, 0]
inputs.generation_cost = cost

linear_flow!(model, inputs)
generation = model.var["Generation"]
@objective(model.optimizer, Min, sum(cost .* generation))
optimize!(model.optimizer)

results = DCOPFResults()
save_all_results!(model, results, 1)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
11 rows, 4 cols, 20 nonzeros
0 rows, 0 cols, 0 nonzeros
Presolve : Reductions: rows 0(-21); columns 0(-6); elements 0(-36) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     :  5.5769230769e-01
HiGHS run time      :          0.00


In [18]:
println("===================================")
print("Geração (pu): ")
results.var["Generation", 1] |> display
println("===================================")

print("Fluxo de potência (pu): ")
results.expr["Flow", 1] |> display
println("===================================")

print("Perdas nas linhas (pu): ")
results.expr["PowerLoss", 1] |> display
println("===================================")


Geração (pu): 

3-element Vector{Float64}:
 0.48461538461538467
 0.31538461538461543
 0.0

Fluxo de potência (pu): 

3×3 Matrix{Float64}:
  0.0        0.1       0.384615
 -0.1        0.0       0.415385
 -0.384615  -0.415385  0.0

Perdas nas linhas (pu): 

3×3 Matrix{Float64}:
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

### Caso com perdas habilitadas, mas linhas sem perdas

In [20]:
### sanity check - with losses and no real part in Ybus, loss must be zero
model = DCOPFModel()

inputs = DCOPFInputs()
inputs.Ybus = [ 22.5im -10im -12.5im; -10im 30im -20im; -12.5im -20im 32.5im ]
inputs.demand = [0, 0, .8]
inputs.min_generation = zeros(3)
inputs.max_generation = [.5, .5, 0]
max_flow = [0 .1 .5; .1 0 .5; .5 .5 0]
inputs.min_flow = -max_flow
inputs.max_flow = max_flow
cost = [.5, 1, 0]
inputs.generation_cost = cost
max_iteration = 10
inputs.max_iteration = max_iteration
inputs.consider_losses = true
inputs.tolerance = 1e-5

results = DCOPFResults()

for iteration in 1:max_iteration
    @info "Iteration $iteration"
    model.optimizer = Model(HiGHS.Optimizer)
    linear_flow!(model, inputs)
    generation = model.var["Generation"]
    @objective(model.optimizer, Min, sum(cost .* generation))
    optimize!(model.optimizer)
    save_all_results!(model, results, iteration)
    new_cut = new_loss_cut!(model, inputs, results, iteration)
    if new_cut == :stop
        @info "Optimal found"
        break
    end
end

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
11 rows, 7 cols, 23 nonzeros
4 rows, 6 cols, 12 nonzeros
0 rows, 0 cols, 0 nonzeros
Presolve : Reductions: rows 0(-21); columns 0(-12); elements 0(-42) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     :  5.5769230769e-01
HiGHS run time      :          0.00


┌ Info: Iteration 1
└ @ Main In[20]:22
┌ Info: Optimal found
└ @ Main In[20]:31


In [21]:
println("===================================")
print("Geração (pu): ")
results.var["Generation", 1] |> display
println("===================================")

print("Fluxo de potência (pu): ")
results.expr["Flow", 1] |> display
println("===================================")

print("Perdas nas linhas (pu): ")
results.expr["PowerLoss", 1] |> display
println("===================================")


Geração (pu): 

3-element Vector{Float64}:
 0.48461538461538467
 0.31538461538461543
 0.0

Fluxo de potência (pu): 

3×3 Matrix{Float64}:
  0.0        0.1       0.384615
 -0.1        0.0       0.415385
 -0.384615  -0.415385  0.0

Perdas nas linhas (pu): 

3×3 Matrix{Float64}:
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

### Caso considerando perdas nas linhas de transmissão

In [23]:
model = DCOPFModel()

inputs = DCOPFInputs()
inputs.Ybus = ([ 0.05 - 0.65im -0.04 + 0.4im -0.01 + 0.25im;
                -0.04 + 0.4im 0.06 - 0.6im -0.02 + 0.2im;
                -0.01 + 0.25im -0.02 + 0.2im 0.03 - 0.45im
                ]).^-1
inputs.demand = [0, 0, .8]
inputs.min_generation = zeros(3)
inputs.max_generation = [.5, .5, 0]
max_flow = [0 .1 .5; .1 0 .5; .5 .5 0]
inputs.min_flow = -max_flow
inputs.max_flow = max_flow
cost = [.5, 1, 0]
inputs.generation_cost = cost
max_iteration = 10
inputs.max_iteration = max_iteration
inputs.consider_losses = true
inputs.tolerance = 1e-8

results = DCOPFResults()

for iteration in 1:max_iteration
    @info "Iteration $iteration"
    model.optimizer = Model(HiGHS.Optimizer)
    linear_flow!(model, inputs)
    generation = model.var["Generation"]
    @objective(model.optimizer, Min, sum(cost .* generation))
    optimize!(model.optimizer)
    save_all_results!(model, results, iteration)
    new_cut = new_loss_cut!(model, inputs, results, iteration)
    if new_cut == :stop
        @info "Optimal found"
        break
    end
end

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
11 rows, 7 cols, 23 nonzeros
4 rows, 7 cols, 13 nonzeros
4 rows, 7 cols, 13 nonzeros
Presolve : Reductions: rows 4(-17); columns 7(-5); elements 13(-29)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -1.1374736092e-07 Pr: 3(3.25) 0s
          3     5.5000000000e-01 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 3
Objective value     :  5.5000000000e-01
HiGHS run time      :          0.03


┌ Info: Iteration 1
└ @ Main In[23]:24
┌ Info: Iteration 2
└ @ Main In[23]:24


Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
17 rows, 10 cols, 40 nonzeros
10 rows, 10 cols, 30 nonzeros
10 rows, 10 cols, 30 nonzeros
Presolve : Reductions: rows 10(-20); columns 10(-2); elements 30(-30)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -1.0465400550e-08 Pr: 3(0.8125) 0s
          9     5.5485657639e-01 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 9
Objective value     :  5.5485657639e-01
HiGHS run time      :          0.00
Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
23 rows, 10 cols, 54 nonzeros
16 rows, 10 cols, 44 nonzeros
16 rows, 10 cols, 44 nonzeros
Presolve : Reductions: rows 16(-23); columns 10(-2); elements 44(-34)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infe

┌ Info: Iteration 3
└ @ Main In[23]:24
┌ Info: Optimal found
└ @ Main In[23]:33


In [25]:
println("===================================")
print("Geração (pu): ")
results.var["Generation", 3] |> display
println("===================================")

print("Fluxo de potência (pu): ")
results.expr["Flow", 3] |> display
println("===================================")

print("Perdas nas linhas (pu): ")
results.expr["PowerLoss", 3] |> display
println("===================================")

Geração (pu): 

3-element Vector{Float64}:
 0.5
 0.3048567229073016
 0.0

Fluxo de potência (pu): 

3×3 Matrix{Float64}:
  0.0         0.0753919  0.4236
 -0.0753919   0.0        0.378716
 -0.4236     -0.378716   0.0

Perdas nas linhas (pu): 

3×3 Matrix{Float64}:
 0.0          0.000225106  0.0017915
 0.000225106  0.0          0.00284011
 0.0017915    0.00284011   0.0

In [50]:
plotly()

x = LinRange(-.1,.1,100)
tl = zeros(100);
for i in eachindex(x)                                                                                             
    tl[i] = transmission_loss(x[i], -real(inputs.Ybus[3,1]))                                                 
end
plt = plot(x, tl, label = "Perdas originais (quadrática)")
for i in 2:3
    v = model.cuts[("PowerLoss", i-1)][3,1]
    reshape(v, length(v), 1)
    y = [x ones(100)] * v
    plt = plot!(x, y, label = "Corte iteração $i")
end
plt = plot!(legend = :outertopright, ylabel = "Perdas (pu)", xlabel = "Fase (rad)")
plt = plot!(title = "Linha 1-3", size = (800, 400))

In [52]:
plotly()

x = LinRange(-.1,.1,100)
tl = zeros(100);
for i in eachindex(x)                                                                                             
    tl[i] = transmission_loss(x[i], -real(inputs.Ybus[2,1]))                                                 
end
plt = plot(x, tl, label = "Perdas originais (quadrática)")
for i in 2:3
    v = model.cuts[("PowerLoss", i-1)][2,1]
    reshape(v, length(v), 1)
    y = [x ones(100)] * v
    plt = plot!(x, y, label = "Corte iteração $i")
end
plt = plot!(legend = :outertopright, ylabel = "Perdas (pu)", xlabel = "Fase (rad)")
plt = plot!(title = "Linha 1-2", size = (800, 400))

In [53]:
plotly()

x = LinRange(-.1,.1,100)
tl = zeros(100);
for i in eachindex(x)                                                                                             
    tl[i] = transmission_loss(x[i], -real(inputs.Ybus[3,2]))                                                 
end
plt = plot(x, tl, label = "Perdas originais (quadrática)")
for i in 2:3
    v = model.cuts[("PowerLoss", i-1)][3,2]
    reshape(v, length(v), 1)
    y = [x ones(100)] * v
    plt = plot!(x, y, label = "Corte iteração $i")
end
plt = plot!(legend = :outertopright, ylabel = "Perdas (pu)", xlabel = "Fase (rad)")
plt = plot!(title = "Linha 2-3", size = (800, 400))